In [1]:
using DataFrames
using CSV
using DelimitedFiles
using Test
using Pkg
Pkg.activate(".")
Pkg.instantiate()

  Activating project at `d:\SeqFISH+Processing_project\UntanglingBarcodes\codebook_generation\get_RS_codebooks\local`


# Overview of Notebook
This notebook shows how to get shortened Reed-Solomon codes. This will allow users more flexibility by allowing access to Reed-Solomon codes that encode fewer codewords with fewer hybridizations while maintaining maximum distance separability at a desired minimum distance. A shortened Reed

In [2]:
function get_num_codewords_of_weight_vn(q,n,w,d)
    i = collect(0:(w-d))
    (q-1)*binomial(n,w)*sum(((-1) .^ i) .* (binomial.(w-1, i)) .* (q .^ ((w - d) .- i)))
end

get_num_codewords_of_weight_vn (generic function with 1 method)

In [3]:
full_cb = DataFrame(CSV.File("output/RS_q11_k7_w4cb.csv", header=false))
full_H = readdlm("output/RS_q11_k7_H.csv", ',')
init_n = size(full_H)[2]

10

## Set desired number of symbols in shortened code

In [4]:
final_n = 9

9

In [5]:
full_cb

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,1,0,0,4,0,0,0,0,9,1
2,0,0,10,0,3,0,0,0,9,1
3,0,4,0,0,0,0,0,2,9,1
4,5,0,9,0,0,0,0,0,10,1
5,0,3,0,3,0,0,0,0,10,1
6,0,0,0,0,10,9,0,0,10,1
7,0,1,0,0,0,0,7,4,0,1
8,0,0,0,7,0,2,0,4,0,1
9,0,9,0,0,8,0,0,5,0,1


In [6]:
full_H

3×10 Matrix{Float64}:
 2.0  4.0  8.0  5.0  10.0  9.0  7.0  3.0  6.0  1.0
 4.0  5.0  9.0  3.0   1.0  4.0  5.0  9.0  3.0  1.0
 8.0  9.0  6.0  4.0  10.0  3.0  2.0  5.0  7.0  1.0

In [7]:
size(full_H)[2]

10

In [8]:
init_n = size(full_H)[2]
final_n = 9

9

In [9]:
sum(prod(Matrix(full_cb[:,(final_n+1):end]) .== 0, dims=2))

1260

In [10]:
keepers = reshape(prod(Matrix(full_cb[:,(final_n+1):end]) .== 0, dims=2), nrow(full_cb))
@test sum(keepers) == get_num_codewords_of_weight_vn(11,9,4,4)

Test Passed

In [11]:
shortened_cb = full_cb[findall(i -> i == 1, keepers), :]
@test all(shortened_cb.Column10 .== 0)
shortened_cb = select(shortened_cb, Not(:Column10))

Row,Column1,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9
,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,0,10,0,3,0,0,0,9,1
2,4,0,0,0,0,0,2,9,1
3,3,0,3,0,0,0,0,10,1
4,0,0,0,10,9,0,0,10,1
5,1,0,0,0,0,7,4,0,1
6,0,0,7,0,2,0,4,0,1
7,9,0,0,8,0,0,5,0,1
8,0,7,0,0,3,0,5,0,1
9,3,0,0,0,10,0,6,0,1


In [12]:
CSV.write("output/q11n9md4w4.csv", shortened_cb)

"output/q11n9md4w4.csv"

In [13]:
full_H

3×10 Matrix{Float64}:
 2.0  4.0  8.0  5.0  10.0  9.0  7.0  3.0  6.0  1.0
 4.0  5.0  9.0  3.0   1.0  4.0  5.0  9.0  3.0  1.0
 8.0  9.0  6.0  4.0  10.0  3.0  2.0  5.0  7.0  1.0

In [14]:
shortened_H = full_H[:, 1:end-1]

3×9 Matrix{Float64}:
 2.0  4.0  8.0  5.0  10.0  9.0  7.0  3.0  6.0
 4.0  5.0  9.0  3.0   1.0  4.0  5.0  9.0  3.0
 8.0  9.0  6.0  4.0  10.0  3.0  2.0  5.0  7.0

In [15]:
open("output/q11_n9_md4_H.csv", "w") do io
    writedlm(io, shortened_H,",")
end